# Importing needed packages

In [1]:
import pprint
import re
import unicodedata

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
pp = pprint.PrettyPrinter(indent=1, width=140, compact=True)

In [3]:
cleaner_map = {"\t": "", "\n": "", "\r": ""}
translator = str.maketrans(cleaner_map)

In [4]:
def preprocess_string(str_: str, translator: dict) -> str:
    str_ = unicodedata.normalize("NFKD", str_)
    str_ = str_.translate(translator)
    str_ = " ".join(str_.split())

    return str_

In [5]:
def has_numbers(s):
    return bool(re.search(r"\d", s))

In [6]:
def return_score(review):
    scores = []
    score = "N/A"
    for s in review.split("<p>")[-3:]:
        if has_numbers(s) and (("/" in s) or ("из" in s)):
            scores.append(s)

    score_candidate = BeautifulSoup(scores[-1]).get_text() if scores else "N/A"
    if scores and len(score_candidate.split(" ")) <= 4 and score_candidate != "N/A":
        score = score_candidate
    return score

# Parsing main page

Information, that should be parsed:
 - Film ID
 - Russian title
 - Original title
 - Everything in the section "about the film"
 - Kinopoisk average score & the number of ratings
 - IMDB average score & the number of ratings
 - Film synopsis
 - Film critics rating in the world
     - Percentage
     - Number of reviews
     - Star rating
 - Film critics rating in Russia
     - Percentage
     - Number of reviews

## Getting the code from the downloaded page

In [7]:
path_front = "./data/indiana_front.html"
with open(path_front, "r", encoding="utf-8") as f:
    indiana_front = f.read()

Passing the code into the `BeautifulSoup` constructor

In [9]:
soup_front = BeautifulSoup(indiana_front, "html.parser")

In [10]:
type(soup_front)

bs4.BeautifulSoup

In [98]:
movie_dict = {}

## Parsing movie's id

In [99]:
movie_id_dict = {}

In [100]:
film_id = soup_front.find_all(
    "link", attrs={"href": re.compile("https://www.kinopoisk.ru/film/")}
)[0]

In [101]:
film_id_ = film_id["href"].split("/")[-2]
film_id_

'339'

In [102]:
movie_id_dict["id"] = film_id_

## Parsing movie's name

In [103]:
movie_name_dict = {}

In [104]:
movie_name = soup_front.find_all("h1", attrs={"class": re.compile("styles_title")})[0]

In [105]:
movie_name_ = movie_name.get_text()
movie_name_

'Индиана Джонс: В поисках утраченного ковчега (1981)'

In [106]:
movie_name_original = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_originalTitle")}
)[0]

In [107]:
movie_name_original_ = movie_name_original.get_text()
movie_name_original_

'Raiders of the Lost Ark'

In [108]:
movie_name_dict["Название"] = movie_name_
movie_name_dict["Оргинальное название"] = movie_name_original_

## Parsing information about the movie

In [109]:
movie_info_dict = {}

In [110]:
movie_info_divs = soup_front.find_all(attrs={"data-test-id": "encyclopedic-table"})[0]

In [111]:
values = []
for div in movie_info_divs.find_all(
    "div", attrs={"class": re.compile("styles_value"), "data-tid": re.compile(".*")}
):
    current_value = div.get_text().replace("сборы", "")
    if "слова" not in current_value:
        values.append(current_value)

In [112]:
titles = []
for div in movie_info_divs.find_all(class_=re.compile("styles_title")):
    current_title = div.get_text()
    titles.append(current_title)

In [113]:
assert len(values) == len(titles)
movie_info_dict = dict(zip(titles, values))
movie_dict = {**movie_name_dict, **movie_info_dict}

In [114]:
movie_dict

{'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Год производства': '1981',
 'Страна': 'США',
 'Жанр': 'приключения, боевик, комедия',
 'Слоган': '«Indiana Jones - the new hero from the creators of JAWS and STAR WARS»',
 'Режиссер': 'Стивен Спилберг',
 'Сценарий': 'Лоуренс Кэздан, Джордж Лукас, Филип Кауфман',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Оператор': 'Дуглас Слоком',
 'Композитор': 'Джон Уильямс',
 'Художник': 'Норман Рейнольдс, Айвор Лесли Дилли, Дебора Нэдулмэн, ...',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Бюджет': '$18\xa0000\xa0000',
 'Сборы в США': '$212\xa0222\xa0025',
 'Сборы в мире': '+ $141\xa0766\xa0000 = $353\xa0988\xa0025',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Премьера в мире': '12 июня 1981, ...',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Возраст': '12+',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Время': '115 мин.

## Parsing movie's rating

In [115]:
movie_rating_dict = {}

In [116]:
movie_rating_kinopoisk = soup_front.find_all(
    "a", attrs={"class": re.compile("film-rating-value")}
)[0]

In [117]:
movie_rating_kinopoisk_ = movie_rating_kinopoisk.get_text()
movie_rating_kinopoisk_

'8.0'

In [118]:
movie_rating_count_kinopoisk = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_count")}
)[0]

In [119]:
movie_rating_count_kinopoisk_ = movie_rating_count_kinopoisk.get_text()
movie_rating_count_kinopoisk_

'142 538 оценок'

In [120]:
movie_rating_imdb = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_valueSection")}
)[0]

In [121]:
movie_rating_imdb_ = movie_rating_imdb.get_text()
movie_rating_imdb_

'IMDb: 8.40'

In [122]:
movie_rating_count_imdb = soup_front.find_all(
    "div", attrs={"class": re.compile("film-sub-rating")}
)[0].find("span", attrs={"class": re.compile("styles_count")})

In [123]:
movie_rating_count_imdb_ = movie_rating_count_imdb.get_text()
movie_rating_count_imdb_

'938 581 оценка'

In [124]:
movie_rating_dict["Рейтинг кинопоиска"] = movie_rating_kinopoisk_
movie_rating_dict["Количество оценок кинопоиска"] = movie_rating_count_kinopoisk_
movie_rating_dict["Рейтинг IMDb"] = movie_rating_imdb_
movie_rating_dict["Количество оценок IMDb"] = movie_rating_count_imdb_

In [125]:
movie_rating_dict

{'Рейтинг кинопоиска': '8.0',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Количество оценок IMDb': '938 581 оценка'}

## Parsing synopsis

In [126]:
movie_synopsis_dict = {}

In [127]:
movie_synopsis = soup_front.find_all(
    "div", attrs={"class": re.compile("styles_filmSynopsis")}
)[0]

In [128]:
movie_synopsis_ = unicodedata.normalize("NFKD", movie_synopsis.get_text())
movie_synopsis_

'Известный археолог и специалист по оккультным наукам доктор Джонс получает опасное задание от правительства США. Он должен отправиться на поиски уникальной реликвии — священного Ковчега. Но Индиана и не подозревает, что аналогичный приказ уже получили тайные агенты Адольфа Гитлера.'

In [129]:
movie_synopsis_dict = {"Синопсис": movie_synopsis_}

## Parsing critics rating

In [130]:
movie_critics_rating_dict = {}

In [131]:
movie_world_critics = soup_front.find_all(
    "div", attrs={"class": re.compile("styles_ratingBar")}
)[0]

In [132]:
movie_world_critics_percentage = movie_world_critics.find_all(
    "span", attrs={"class": re.compile("film-rating-value")}
)[0]

In [133]:
movie_world_critics_percentage_ = movie_world_critics_percentage.get_text()
movie_world_critics_percentage_

'95%'

In [134]:
movie_world_critics_number_of_reviews = movie_world_critics.find_all(
    "span", attrs={"class": re.compile("styles_count")}
)[0]

In [135]:
movie_world_critics_number_of_reviews_ = (
    movie_world_critics_number_of_reviews.get_text()
)
movie_world_critics_number_of_reviews_

'86 оценок9.3'

In [136]:
movie_world_critics_star_value = movie_world_critics.find_all(
    "span", attrs={"class": re.compile("styles_starValue")}
)[0]

In [137]:
movie_world_critics_star_value_ = movie_world_critics_star_value.get_text()
movie_world_critics_star_value_

'9.3'

In [138]:
def right_strip_trailing(original, trailing):
    trailing_len = len(trailing)
    if original[-trailing_len:] == trailing:
        return original[:-trailing_len]
    else:
        return original
    return original

In [139]:
movie_world_critics_number_of_reviews_ = right_strip_trailing(
    movie_world_critics_number_of_reviews_, movie_world_critics_star_value_
)
movie_world_critics_number_of_reviews_

'86 оценок'

In [140]:
movie_russian_critics = soup_front.find_all(
    "div", attrs={"class": re.compile("styles_ratingBar")}
)[2]

In [141]:
movie_russian_critics_percentage = movie_russian_critics.find_all(
    "span", attrs={"class": re.compile("film-rating-value")}
)[0]

In [142]:
movie_russian_critics_percentage_ = movie_russian_critics_percentage.get_text()
movie_russian_critics_percentage_

'–'

In [143]:
movie_russian_critics_number_of_reviews = movie_russian_critics.find_all(
    "span", attrs={"class": re.compile("styles_count")}
)[0]

In [144]:
movie_russian_critics_number_of_reviews_ = (
    movie_russian_critics_number_of_reviews.get_text()
)
movie_russian_critics_number_of_reviews_

'1 оценка'

In [145]:
movie_critics_rating_dict = {
    "Средняя полярность международных критиков": movie_world_critics_percentage_,
    "Количество рецензий международных критиков": movie_world_critics_number_of_reviews_,
    "Средняя оценка международных критиков": movie_world_critics_star_value_,
    "Средняя полярность российских критиков": movie_russian_critics_percentage_,
    "Количество рецензий российских критиков": movie_russian_critics_number_of_reviews_,
}
movie_critics_rating_dict

{'Средняя полярность международных критиков': '95%',
 'Количество рецензий международных критиков': '86 оценок',
 'Средняя оценка международных критиков': '9.3',
 'Средняя полярность российских критиков': '–',
 'Количество рецензий российских критиков': '1 оценка'}

# Preparing final dict with information about the film

In [149]:
movie_dict = {
    **movie_id_dict,
    **movie_name_dict,
    **movie_info_dict,
    **movie_rating_dict,
    **movie_synopsis_dict,
    **movie_critics_rating_dict,
}

movie_dict_cleaned = {
    key: preprocess_string(value, translator) for key, value in movie_dict.items()
}

pp.pprint(movie_dict_cleaned)

{'id': '339',
 'Бюджет': '$18 000 000',
 'Возраст': '12+',
 'Время': '115 мин. / 01:55',
 'Год производства': '1981',
 'Жанр': 'приключения, боевик, комедия',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Количество оценок IMDb': '938 581 оценка',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Количество рецензий международных критиков': '86 оценок',
 'Количество рецензий российских критиков': '1 оценка',
 'Композитор': 'Джон Уильямс',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оператор': 'Дуглас Слоком',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Премьера в мире': '12 июня 1981, ...',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Режиссер': 'Стивен Спилберг',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Рейтинг кинопоиска': '8.0',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Сборы в США': '$212 222 025',
 'Сборы в м

# Parsing page with reviews

Information, that should be parsed for every review:
 - Username of reviewer
 - Reviewer ID
 - Date and time of review
 - Review sentiment
 - Review subtitle (if present)
 - Review body (preserving the division into paragraphs)
 - Review score (if possible - the last paragraph of the review)
 - The usefulness of review ratio
 - Direct link to the review

## Getting the code for the page with movie

In [3]:
# f = requests.get("https://www.kinopoisk.ru/film/339/reviews/")

## Getting the code from the downloaded page

In [153]:
path_reviews = "./data/indiana_reviews.html"
with open(path_reviews, "r", encoding="utf-8") as f:
    indiana_reviews = f.read()

Passing the code into the `BeautifulSoup` constructor

In [154]:
soup_reviews = BeautifulSoup(indiana_reviews, "html.parser")

In [155]:
review_dict = {}

## Parsing movie's id

Parsing movie id so we can match the reviews with the film

In [216]:
review_id_dict = {}

In [217]:
film_id = soup_reviews.find_all(
    "link", attrs={"href": re.compile("https://www.kinopoisk.ru/film/")}
)[0]

In [218]:
film_id_ = film_id["href"].split("/")[-3]

In [219]:
review_id_dict["ID фильма"] = film_id_
review_id_dict

{'ID фильма': '339'}

## Parsing reviews

In [164]:
review_content_dict = {}

In [165]:
reviews = soup_reviews.find_all("div", attrs={"class": "reviewItem userReview"})

Let's start with parsing single review and then we will use this pattern to parse all the reviews on the page

In [166]:
review = reviews[0]

Let's look at the content of review 

In [167]:
review

<div class="reviewItem userReview" data-id="3128202" id="_DRAGGABLE_3128202" style="width: 555px; position: relative; padding: 1px 0 40px 0">
<div id="3128202"></div>
<div class="response good" id="div_review_3128202" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">
<meta content="Знакомство с Индианой Джонс в 2022." itemprop="headline"/>
<div itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<img alt="" data-lazy-src="https://avatars.mds.yandex.net/get-kino-vod-users-avatar/57335/63366919-43-695797.jpg/46x73_bw" itemprop="image" src="https://st.kp.yandex.net/images/spacer.gif" title="" width="46"/>
<div>
<p class="profile_name"><s></s><a data-popup-info="disabled" href="/user/63366919/" itemprop="name">komisar.jeber</a></p>
<ul class="actions">
<li class="first"><a href="/user/63366919/comments/">рецензии</a></li>
<li><a href="/user/63366919/votes/">оценки</a></li>
<li><a href="/user/63366919/friends/">друзья</a></li>
<li><a href="/user/63366919/m

### Parsing username of reviewer

In [168]:
reviewer_username = review.find_all("p", attrs={"class": "profile_name"})[0]

In [169]:
reviewer_username_ = reviewer_username.get_text()
reviewer_username_

'komisar.jeber'

### Parsing id of reviewer

In [170]:
reviewer_id = reviewer_username.find_all("a", href=True)[0]

In [171]:
reviewer_id_ = reviewer_id["href"].split("/")[-2]
reviewer_id_

'63366919'

### Parsing date & time of review

In [172]:
datetime = review.find_all("span", attrs={"class": "date"})[0]

In [173]:
datetime_ = datetime.get_text().replace("|", "").strip()
datetime_ = " ".join(datetime_.split())
datetime_

'23 февраля 2022 15:18'

### Parsing review sentiment

In [174]:
sentiment = review.find("div", attrs={"class": "response"})

In [175]:
sentiment_ = sentiment.attrs["class"][-1]
sentiment_

'good'

### Parsing review subtitle

In [176]:
subtitle = review.find("p", attrs={"class": "sub_title"})

In [177]:
subtitle_ = subtitle.get_text()
subtitle_

'Знакомство с Индианой Джонс в 2022.'

### Parsing review body

In [178]:
review_body = review.find("span", attrs={"itemprop": "reviewBody"})

In [179]:
review_body_ = review_body.contents
review_body_

['Спустя более 40 лет после выхода данного фильма я впервые его посмотрел. Ранее видел множество фрагментов из него и кучу отсылок. Так каково же это - впервые посмотреть приключения Индианы Джонс в 2022 году?',
 <br/>,
 '\n',
 <br/>,
 '\nДаже спустя 41 год после премьеры, фильм смотрится довольно прилично. Главные его достоинства это экшен и главный герой, хотя искушенному кинолюбителю этого может показаться мало. Фильм смотрится практически на одном дыхании. Экшен изобретательный и снят отлично, а для 81 года просто невообразимо. Его в фильме более чем достаточно, иногда от него даже можно устать, хоть кино и динамичное.',
 <br/>,
 '\n',
 <br/>,
 '\nСам Индиана Джонс в исполнении Харисона Форда получился то что надо, однако на сегодняшний день он выглядит весьма типично и подобных персонажей в кинохронике уже немало.  Все же это не умаляет заслуг создателей и актер в этой роли смотрится превосходно. Кого-то другого на его месте представить сложно.',
 <br/>,
 '\n',
 <br/>,
 '\nОтдельн

I've decided to remove unnecessary elements and add paragraph separator as `<p>`.  
So paragraphs can be easily separated in the future pre-processing.

In [180]:
review_body_final_ = "<p>".join(
    [el.replace("\n", "") for el in review_body_ if str(el) not in ["<br/>", "\n"]]
)
review_body_final_

"Спустя более 40 лет после выхода данного фильма я впервые его посмотрел. Ранее видел множество фрагментов из него и кучу отсылок. Так каково же это - впервые посмотреть приключения Индианы Джонс в 2022 году?<p>Даже спустя 41 год после премьеры, фильм смотрится довольно прилично. Главные его достоинства это экшен и главный герой, хотя искушенному кинолюбителю этого может показаться мало. Фильм смотрится практически на одном дыхании. Экшен изобретательный и снят отлично, а для 81 года просто невообразимо. Его в фильме более чем достаточно, иногда от него даже можно устать, хоть кино и динамичное.<p>Сам Индиана Джонс в исполнении Харисона Форда получился то что надо, однако на сегодняшний день он выглядит весьма типично и подобных персонажей в кинохронике уже немало.  Все же это не умаляет заслуг создателей и актер в этой роли смотрится превосходно. Кого-то другого на его месте представить сложно.<p>Отдельно хочется выделить Карен Ален и ее персонажа в частности. Для стандартной девушки 

### Parsing review score

In [182]:
score_ = return_score(review_body_final_)
score_

'7 из 10'

### Parsing usefulness of review ratio

In [183]:
usefulness_ratio = review.find("li", attrs={"id": re.compile("comment_num_vote")})

In [184]:
usefulness_ratio_ = usefulness_ratio.get_text()
usefulness_ratio_

'1 / 0'

### Parsing direct link to the review

In [185]:
direct_link = review.find("p", attrs={"class": "links"}).find(href=True)

In [186]:
direct_link_ = "https://www.kinopoisk.ru/" + direct_link["href"]
direct_link_

'https://www.kinopoisk.ru//user/63366919/comment/3128202/'

### Preparing dict with information from review

In [187]:
review_content_dict = {
    "Имя пользователя": reviewer_username_,
    "ID пользователя": reviewer_id_,
    "Дата и время отзыва": datetime_,
    "Полярность отзыва": sentiment_,
    "Подзаголовок отзыва": subtitle_,
    "Отзыв": review_body_final_,
    "Оценка из отзыва": score_,
    "Полезность отзыва": usefulness_ratio_,
    "Прямая ссылка на отзыв": direct_link_,
}
review_content_dict

{'Имя пользователя': 'komisar.jeber',
 'ID пользователя': '63366919',
 'Дата и время отзыва': '23 февраля 2022 15:18',
 'Полярность отзыва': 'good',
 'Подзаголовок отзыва': 'Знакомство с Индианой Джонс в 2022.',
 'Отзыв': "Спустя более 40 лет после выхода данного фильма я впервые его посмотрел. Ранее видел множество фрагментов из него и кучу отсылок. Так каково же это - впервые посмотреть приключения Индианы Джонс в 2022 году?<p>Даже спустя 41 год после премьеры, фильм смотрится довольно прилично. Главные его достоинства это экшен и главный герой, хотя искушенному кинолюбителю этого может показаться мало. Фильм смотрится практически на одном дыхании. Экшен изобретательный и снят отлично, а для 81 года просто невообразимо. Его в фильме более чем достаточно, иногда от него даже можно устать, хоть кино и динамичное.<p>Сам Индиана Джонс в исполнении Харисона Форда получился то что надо, однако на сегодняшний день он выглядит весьма типично и подобных персонажей в кинохронике уже немало.  В

### Parsing all reviews from page

In [220]:
reviews_list = []
for i, review in enumerate(reviews):
    reviewer_username = review.find_all("p", attrs={"class": "profile_name"})[0]
    reviewer_username_ = reviewer_username.get_text()

    reviewer_id = reviewer_username.find_all("a", href=True)[0]
    reviewer_id_ = reviewer_id["href"].split("/")[-2]

    datetime = review.find_all("span", attrs={"class": "date"})[0]
    datetime_ = datetime.get_text().replace("|", "").strip()
    datetime_ = " ".join(datetime_.split())

    sentiment = review.find("div", attrs={"class": "response"})
    sentiment_ = sentiment.attrs["class"][-1]

    subtitle = review.find("p", attrs={"class": "sub_title"})
    subtitle_ = subtitle.get_text()

    review_body = review.find("span", attrs={"itemprop": "reviewBody"})
    review_body_ = review_body.contents
    review_body_score_ = "<p>".join(
        [
            str(el).replace("\n", "")
            for el in review_body_
            if str(el) not in ["<br/>", "\n"]
        ]
    )

    score_ = return_score(review_body_score_)
    review_body_final_ = review.find_all("span", attrs={"itemprop": "reviewBody"})[
        0
    ].get_text()
    review_body_final_ = "<p>".join(
        [str(el) for el in review_body_final_.split("\n\n")]
    )

    usefulness_ratio = review.find("li", attrs={"id": re.compile("comment_num_vote")})
    usefulness_ratio_ = usefulness_ratio.get_text()

    direct_link = review.find("p", attrs={"class": "links"}).find(href=True)

    direct_link_ = (
        "https://www.kinopoisk.ru" + direct_link["href"] if direct_link else "N/A"
    )

    review_content_dict = {
        **review_id_dict,
        "Имя пользователя": reviewer_username_,
        "ID пользователя": reviewer_id_,
        "Дата и время отзыва": datetime_,
        "Полярность отзыва": sentiment_,
        "Подзаголовок отзыва": subtitle_,
        "Отзыв": review_body_final_,
        "Оценка из отзыва": score_,
        "Полезность отзыва": usefulness_ratio_,
        "Прямая ссылка на отзыв": direct_link_,
    }

    review_content_dict_cleaned = {
        key: preprocess_string(value, translator)
        for key, value in review_content_dict.items()
    }

    reviews_list.append(review_content_dict_cleaned)

Let's look what we've got

In [221]:
pp.pprint(reviews_list[::10])

[{'ID пользователя': '63366919',
  'ID фильма': '339',
  'Дата и время отзыва': '23 февраля 2022 15:18',
  'Имя пользователя': 'komisar.jeber',
  'Отзыв': 'Спустя более 40 лет после выхода данного фильма я впервые его посмотрел. Ранее видел множество фрагментов из него и кучу '
           'отсылок. Так каково же это - впервые посмотреть приключения Индианы Джонс в 2022 году?<p>Даже спустя 41 год после премьеры, '
           'фильм смотрится довольно прилично. Главные его достоинства это экшен и главный герой, хотя искушенному кинолюбителю этого '
           'может показаться мало. Фильм смотрится практически на одном дыхании. Экшен изобретательный и снят отлично, а для 81 года '
           'просто невообразимо. Его в фильме более чем достаточно, иногда от него даже можно устать, хоть кино и динамичное.<p>Сам '
           'Индиана Джонс в исполнении Харисона Форда получился то что надо, однако на сегодняшний день он выглядит весьма типично и '
           'подобных персонажей в кино

In [222]:
review_df = pd.DataFrame(reviews_list)
review_df.to_csv("reviews.csv", index=False)

In [223]:
review_df.head()

ID фильма   Имя пользователя ID пользователя     Дата и время отзыва  \
0       339      komisar.jeber        63366919   23 февраля 2022 15:18   
1       339  Xiphactinus audax         5973976    12 января 2022 01:00   
2       339      ars-projdakov          426194   29 октября 2020 21:54   
3       339      Владимир Кирк        44080109   10 февраля 2020 16:51   
4       339   nikitla17logunov        34021124  23 сентября 2019 00:02   

  Полярность отзыва                                Подзаголовок отзыва  \
0              good               Знакомство с Индианой Джонс в 2022.   
1              good  Главное — не цель, а то, что случается по пути...   
2              good                                                      
3              good           Куртка, шляпа и кнут - вот мои документы   
4              good  В поисках не утраченного прародителя приключений   

                                               Отзыв Оценка из отзыва  \
0  Спустя более 40 лет после выхода данного фильм...          7 из 10   
1  Стивен Спилберг и Джордж Лукас, отдыхая однажд...          8 из 10   
2  В детстве многие мечтали о приключениях по мир...              N/A   
3  'Индиана Джонс: В поисках утраченного Ковчега'...              N/A   
4  Если кого-нибудь спросить: какой фильм являет...        9,5 из 10   

  Полезность отзыва                             Прямая ссылка на отзыв  
0             1 / 0  https://www.kinopoisk.ru/user/63366919/comment...  
1             2 / 1  https://www.kinopoisk.ru/user/5973976/comment/...  
2             6 / 2  https://www.kinopoisk.ru/user/426194/comment/2...  
3             3 / 3  https://www.kinopoisk.ru/user/44080109/comment...  
4             3 / 2  https://www.kinopoisk.ru/user/34021124/comment...

In [225]:
review_df["Полярность отзыва"].values

array(['good', 'good', 'good', 'good', 'good', 'good', 'neutral', 'good',
       'good', 'good', 'neutral', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'neutral', 'good', 'neutral',
       'good', 'good', 'neutral', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'neutral',
       'good', 'good', 'bad', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'neutral', 'good', 'good',
       'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good',
       'good', 'good', 'good', 'good'], dtype=object)